In [30]:
!pip install youtube_transcript_api

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.


In [1]:
from youtube_transcript_api import YouTubeTranscriptApi
import pandas as pd
from os import listdir
import csv
import time

In [2]:
def write_comments_to_file(video, file_name):
    subs = []
    retry = 0

    while len(subs) == 0 and retry < 10:
        try:
            subs=YouTubeTranscriptApi.get_transcript(video)
            retry += 1
            #print('Trying: ' + str(retry))
        except:
            retry += 1
            #print('Retrying: ' + str(retry))

    status = 0
    phrase_count = 0
    phrase = ""
    
    for text in subs:
        oneLine = text['text'].replace("\n", " ")
        oneLine = " ".join(oneLine.split())
        if len(oneLine) > 0 and oneLine[-1] =='.':
            status = 0
            if len(phrase) > 0:
                phrase = (phrase + " " + oneLine.strip())
            else:
                phrase = oneLine.strip()

            with open("subtitles/" + file_name + '.csv', mode='a') as out_file:
                result_writer = csv.writer(out_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                result_writer.writerow([video, phrase_count, phrase])

            phrase_count += 1
            phrase = ""
        else:
            status = 1
            if len(phrase) > 0:
                phrase = phrase + " " + oneLine.strip()
            else:
                phrase = oneLine.strip()

    # A status of 0 means that the phrase has been building, but was not written to the file
    if status == 0:
        with open("subtitles/" + file_name + '.csv', mode='a') as out_file:
            result_writer = csv.writer(out_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            result_writer.writerow([video, phrase_count, phrase])
    with open("subtitles/summary_" + file_name + '.csv', mode='a') as out_file:
        result_writer = csv.writer(out_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        result_writer.writerow([video, phrase_count])
    #print('Extracted subtitles: ' + str(phrase_count))

In [3]:
def write_subtitles_to_file(video, file_name):
    subs = []
    retry = 0
    phrase = ""

    while len(subs) == 0 and retry < 5:
        try:
            subs=YouTubeTranscriptApi.get_transcript(video)
            retry += 1
        except:
            retry += 1

    for text in subs:
        oneLine = text['text'].replace("\n", " ")
        oneLine = " ".join(oneLine.split())

        if len(phrase) > 0:
            phrase = phrase + " " + oneLine.strip()
        else:
            phrase = oneLine.strip()

    phrase_count = len(phrase)
    
    #video_record = df.iloc[df[df['video']==video].index[0]].to_list()
    video_record = df[df['video']==video].values[0].tolist()
    video_record.append(phrase)
    #print('About to write with phrase_count:', phrase_count, 'for', video)
    if phrase_count > 0:
        with open("subtitles/" + file_name + '.csv', mode='a') as out_file:
            result_writer = csv.writer(out_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            result_writer.writerow(video_record)
        #print('Extracted subtitles for:', video, str(phrase_count))

In [4]:
filepaths = ["subtitles/" + f for f in listdir("subtitles/") if f.endswith('.csv')]
df_subs = pd.concat(map(pd.read_csv, filepaths))
df_subs.shape

(21248, 22)

In [5]:
pd.set_option('display.max_rows', None)
filepaths = ["results/completed/" + f for f in listdir("./results/completed") if f.endswith('.csv')]
df = pd.concat(map(pd.read_csv, filepaths))
df.shape

(17721, 18)

In [6]:
df=df[-df["video"].isin(df_subs['video'])]
df.shape

(2856, 18)

In [7]:
df['video'].unique().shape

(808,)

In [47]:
date = time.strftime('%Y-%m-%d_%H_%M_%S')
file_name = 'subtitles_'+ date
header=df.columns.to_list()
header.append('subtitles')
header

print("File: " + file_name)
counter = 0
with open("subtitles/" + file_name + '.csv', mode='w') as out_file:
    result_writer = csv.writer(out_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    result_writer.writerow(header)

for v in df['video'].unique():
    write_subtitles_to_file(v, file_name)
    #write_comments_to_file(v, file_name)
    if counter%1000 == 0:
        print('Reached: ' + str(counter) + ' ' + time.strftime('%Y-%m-%d_%H_%M_%S'))
    counter += 1

File: subtitles_2021-06-12_06_53_10
Reached: 0 2021-06-12_06_53_12
Reached: 1000 2021-06-12_07_21_57
